## Visualize image-specific class saliency with backpropagation

---

A quick demo of creating saliency maps for CNNs using [FlashTorch 🔦](https://github.com/MisaOgura/flashtorch).


❗This notebook is for those who are using this notebook in **Google Colab**.

If you aren't on Google Colab already, please head to the Colab version of this notebook **[here](https://colab.research.google.com/github/MisaOgura/flashtorch/blob/master/examples/visualise_saliency_with_backprop_colab.ipynb)** to execute.

---

The gradients obtained can be used to visualise an image-specific class saliency map, which can gives some intuition on regions within the input image that contribute the most (and least) to the corresponding output.

More details on saliency maps: [Deep Inside Convolutional Networks: Visualising Image Classification Models and Saliency Maps](https://arxiv.org/pdf/1312.6034.pdf).

### 0. Set up

A GPU runtime is available on Colab for free, from the Runtime tab on the top menu bar.

It is highly recommended to use GPU as a runtime for the enhanced speed of computation.

In [ ]:
# Install flashtorch

!pip install flashtorch

In [ ]:
# Download the example image

!mkdir -p images

!wget https://github.com/MisaOgura/flashtorch/raw/master/examples/images/great_grey_owl.jpg -P /content/images

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

import torch
import torchvision.models as models

from flashtorch.utils import (apply_transforms,
                              denormalize,
                              format_for_plotting,
                              load_image)

from flashtorch.utils import ImageNetIndex

from flashtorch.saliency import Backprop

### 1. Load an image 

In [ ]:
image = load_image('/content/images/great_grey_owl.jpg')

plt.imshow(image)
plt.title('Original image')
plt.axis('off');

### 2. Load a pre-trained Model

In [ ]:
model = models.alexnet(pretrained=True)

### 3. Create an instance of Backprop with the model

In [ ]:
backprop = Backprop(model)

### 4. Calculate the gradients of a target class w.r.t the input image

By default, we return the gradients of all the colour channel.

You can also specify to return a max gradients across the colour channel via `take_max=True` flag, as this was what the authors did in the [paper](https://arxiv.org/pdf/1312.6034.pdf) and sometimes it renders better for visualization.

In [ ]:
imagenet = ImageNetIndex()
target_class = imagenet['great grey owl']

input_ = apply_transforms(image)

# Calculate the gradients of each pixel w.r.t. the input image

gradients = backprop.calculate_gradients(input_, target_class)

# Or, take the maximum of the gradients for each pixel across colour channels.

max_gradients = backprop.calculate_gradients(input_, target_class, take_max=True)

print('Shape of the gradients:', gradients.shape)
print('Shape of the max gradients:', max_gradients.shape)

### 5. Visualize the input image and gradients side-by-side

In [ ]:
backprop.visualize(input_, target_class)

### 6. Visualize with _guided_ backprogation 

In [ ]:
backprop.visualize(input_, target_class, guided=True)